In [17]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [18]:
client = dataiku.api_client()

In [19]:
project = client.get_project("DSPIPELINEFRAUD")

In [26]:
project_code_env = project.get_settings().get_raw()['settings']['codeEnvs']['python']['envName']

In [27]:
project_code_env

'code_env_cc_fraud'

In [28]:
project_recipes = project.list_recipes()
project_python_recipe = [recipe for recipe in project_recipes if recipe['type'] == 'python'] 

In [29]:
python_recipe_code_env = [code_env['params']['envSelection']['envName'] for python_recipe in project_python_recipe if project_python_recipe['params']['envSelection']['envMode'] == 'EXPLICIT_ENV']

[{'name': 'compute_to_delete',
  'projectKey': 'DSPIPELINEFRAUD',
  'type': 'python',
  'neverRecomputeExistingPartitions': False,
  'optionalDependencies': False,
  'params': {'envSelection': {'envMode': 'EXPLICIT_ENV',
    'envName': 'Python36'},
   'containerSelection': {'containerMode': 'INHERIT'},
   'skipPrerunValidate': False},
  'customMeta': {'kv': {}},
  'redispatchPartitioning': False,
  'maxRunningActivities': 0,
  'variables': {},
  'dkuProperties': [],
  'inputs': {'main': {'items': [{'ref': 'transactions_reward_value_by_card_id_prepared',
      'deps': []}]}},
  'outputs': {'main': {'items': [{'ref': 'to_delete', 'appendMode': False}]}},
  'labels': [],
  'versionTag': {'versionNumber': 2,
   'lastModifiedBy': {'login': 'admin'},
   'lastModifiedOn': 1655310512814},
  'creationTag': {'versionNumber': 0,
   'lastModifiedBy': {'login': 'admin'},
   'lastModifiedOn': 1655310500210},
  'tags': [],
  'customFields': {},
  'checklists': {'checklists': []}}]

In [0]:
# list_jupyter_notebooks
# list_recipes
# list_running_notebooks